# Answer Set Programming (ASP) for Fleet Optimization 
By Florian Jauernig & Nicole Gogol


## 1. Problem description, dataset selection

In a highly influential work by Hane et al. (1995), the "fleet assignment problem", which is about determining which _type of aircraft_ to assign to which flight, is presented alongside a possible solution. The problem is multifaceted as it is based on several variables, including profit, aircraft specifications and travel demand. Overall, it can be regarded as an optimization problem since it aims at determining the the most cost-effective and efficient way of assigning aircrafts.

Current research focuses on discovering new ways of solving this problem, i.e. through machine learning algorithms. Multiple attempts to solve variations of this problem have already been published, for example by Grönkvist (2005) who tackled the "tail assignment problem" of assigning _individual aircraft_ to flights. According to Barlow (2019), the airline scheduling problem still has not been solved completely (as of 4 years ago).

The used data was taken from Clarke (2012). Another example case was published by Belobaba (2014). Incidentally, the data tables of the latter example case are presented in basically the same exact structure, meaning that our guess and check program would also work for it - just the hard-coded registrations in the section "Count the number of flights on a route per aircraft" would have to be changed (and the data would have to be changed out, of course).

To use our code, no additional files need to be present apart from this notebook, as the .lp files can be generated by the code. Nevertheless, we included all of the generated files in the submission.

<b>Sources:</b>

Barlow, J. (2019, April 3). Exploring the airline industry’s great unsolved problem: airline scheduling. Amadeus. https://amadeus.com/en/insights/blog/amadeus-optym-airline-scheduling

Belobaba, P. (2014, March 12). Fleet Assignment. Network, Fleet and Schedule Strategic Planning. ITU Aviation Institute. http://aviation.itu.edu.tr/%5Cimg%5Caviation%5Cdatafiles/Lecture%20Notes/Network%20Fleet%20Schedule%20Strategic%20Planning/Lecture%20Notes/12%20-%20Fleet%20Assignment.pdf

Clarke, C. (2012, November 15). Fleet Assignment - Part 1. ProfessorCarlinClarke.com. https://www.professorcarlinclarke.com/avsc-3060-airline-management/fleet-assignment-part-1

Grönkvist, M. (2005). The Tail Assignment Problem. 51st Airline Group of the International Federation of Operational Research Societies Annual Proceedings - Annual Symposium and Study Group Meeting, AGIFORS 2011; Antalya; Turkey; 10 October 2011 Through 14 October 2011, 3, 1509–1804.

Hane, C.A., Barnhart, C., Johnson, E.L. et al. The fleet assignment problem: Solving a large-scale integer program. Mathematical Programming 70, 211–232 (1995). https://doi.org/10.1007/BF01585938


## 2. Problem representation

Taking the requirements of the original task description into account, the given data was encoded in the following tables:

*Table 1 - Flight*: A key aspect here is considering the need for return trips and departure times: every aircraft must return to its origin airport upon returning from a destination, meaning that each aircraft will appear in the schedule at least twice. To account for this, the route number for each origin-destination pair was added with the same route number being used for both directions of travel. This allows for clear identification and tracking of each aircraft's journey.

*Table2 - Aircraft*: This table provides information about the different types of aircrafts available in the fleet, their quantity, capacity and operating cost for flights between Salt Lake City & Atlanta, and Salt Lake City & Phoenix.

*Table3 - Profit*: This table lays out information about the profit generated by using different types of aircrafts for different flights. This information will later be used throughout the entire logic program, as it is essential to maximize the profit in the optimization step.

*Table 4: Fleet*: Additionally, a fleet table including a made-up registration number for each individual aircraft was added. This allows for easy identification and tracking of the respective aircraft being used for each flight and is closer to real-life operational planning than an abstract type-based schedule.

These tables provide the input data that will be used in the subsequent step, namely the logic program. These changes to the problem representation will ensure that all necessary information is captured and considered when creating the aircrafts' daily schedule and calculating the profit. 

In [1]:
%%file data.lp

% Data:
% ====

% Table1: flight(Flight, Route, Origin, Destination, DepartureMST, ArrivalMST, FareUSD, Demand)       
flight("WA 121", 1, "SLC", "ATL", "06:40", "10:25", 275, 200).
flight("WA 122", 1, "SLC", "ATL", "17:00", "22:30", 300, 225).
flight("WA 151", 1, "ATL", "SLC", "11:15", "15:35", 250, 175).
flight("WA 152", 1, "ATL", "SLC", "16:30", "21:00", 325, 150).
flight("WA 153", 2, "PHX", "SLC", "07:45", "09:30", 180, 175).
flight("WA 154", 2, "PHX", "SLC", "18:15", "19:55", 210, 150).
flight("WA 201", 2, "SLC", "PHX", "11:00", "12:40", 175, 125).
flight("WA 202", 2, "SLC", "PHX", "14:20", "16:05", 200, 100).

% Table2: aircraft(Aircraft, Quantity, Capacity, OperatingCostSLC_ATL, OperatingCostSLC_PHX)
aircraft("B737", 2, 150, 30000, 20000). 
aircraft("B757", 1, 216, 42000, 28000). 
aircraft("A318", 1, 120, 24000, 17000). 

% Table3: profit(Flight, Aircraft, Result)
profit("WA 121", "B737", 11250).
profit("WA 121", "B757", 13000).
profit("WA 121", "A318", 9000).

profit("WA 122", "B737", 15000).
profit("WA 122", "B757", 22800).
profit("WA 122", "A318", 12000).

profit("WA 151", "B737", 7500).
profit("WA 151", "B757", 1750).
profit("WA 151", "A318", 6000).

profit("WA 152", "B737", 18750).
profit("WA 152", "B757", 6750).
profit("WA 152", "A318", 15000).

profit("WA 153", "B737", 7000).
profit("WA 153", "B757", 3500).
profit("WA 153", "A318", 4600).

profit("WA 154", "B737", 11500).
profit("WA 154", "B757", 3500).
profit("WA 154", "A318", 8200).

profit("WA 201", "B737", 1875).
profit("WA 201", "B757", -6125).
profit("WA 201", "A318", 4000).

profit("WA 202", "B737", 2000).
profit("WA 202", "B757", -6000).
profit("WA 202", "A318", 5000).

% Table4: fleet(Aircraft, Registration)
fleet("B737", "N737AA").
fleet("B737", "N737BB").
fleet("B757", "N757CC").
fleet("A318", "N318DD").

Overwriting data.lp


## 3. Guess and check 

In the following, a logic program is written in the CLP language, defining the number of predicates and constraints that are used to model a scheduling problem for a fleet of aircraft. It aims at defining the problem and ensuring that the respective conditions are met. Each step executed in the program is specified in the respective line and described in a comment directly above it.

Overall, the aim is to output all practically feasible schedules given the constraints and preventing impossible schedules, e.g. where the same aircraft is being used for two subsequent flights that depart from the same airport without the aircraft returning to this airport first. The program does not consider profitability yet, that will follow in the next step.

In [2]:
%%file logic.lp

% Logic:
% =====
    
% Extract the actual flights from the table:
a_f(Flight) :- flight(Flight, Route, Origin, Destination, DepartureMST, ArrivalMST, FareUSD, Demand).

% Join the fleet table with the profit table
f_p(Flight, Aircraft, Registration, Result) :- fleet(Aircraft, Registration), profit(Flight, Aircraft, Result).

% Guess: For each flight, pick an aircraft:
1{pick(Flight, Aircraft, Registration, Result) : f_p(Flight, Aircraft, Registration, Result) }1 :- a_f(Flight).

% Join picks with flight and aircraft data
p_a(Flight, Route, Aircraft, Registration, Quantity, Origin, Destination, DepartureMST, ArrivalMST, Result) :- pick(Flight, Aircraft, Registration, Result), flight(Flight, Route, Origin, Destination, DepartureMST, ArrivalMST, FareUSD, Demand), aircraft(Aircraft, Quantity, Capacity, OperatingCostSLC_ATL, OperatingCostSLC_PHX).

% Ensure that no aircraft departs on a second flight before its first flight has arrived:
:- DM2 > DM, DM2 < AM, p_a(F,R,A,REG,Q,O,D,DM,AM,RE), p_a(F2,R2,A,REG,Q,O2,D2,DM2,AM2,RE2).

% Ensure that each aircraft returns to its origin airport after flying to its destination [does not work]:
%:- D != O2, p_a(F,R,A,REG,Q,O,D,DM,AM,RE), p_a(F2,R2,A,REG,Q,O2,D2,DM2,AM2,RE2).

% Since this one-liner did not succeed, we took a more complicated 3-step approach:

% Count the number of flights on a route per aircraft:
on_route("N737AA", 1, FlightsOnRoute):- FlightsOnRoute = #count {(F,REG) : REG="N737AA", R=1, p_a(F,R,A,REG,Q,O,D,DM,AM,RE) }.
on_route("N737AA", 2, FlightsOnRoute):- FlightsOnRoute = #count {(F,REG) : REG="N737AA", R=2, p_a(F,R,A,REG,Q,O,D,DM,AM,RE) }.
on_route("N737BB", 1, FlightsOnRoute):- FlightsOnRoute = #count {(F,REG) : REG="N737BB", R=1, p_a(F,R,A,REG,Q,O,D,DM,AM,RE) }.
on_route("N737BB", 2, FlightsOnRoute):- FlightsOnRoute = #count {(F,REG) : REG="N737BB", R=2, p_a(F,R,A,REG,Q,O,D,DM,AM,RE) }.
on_route("N757CC", 1, FlightsOnRoute):- FlightsOnRoute = #count {(F,REG) : REG="N757CC", R=1, p_a(F,R,A,REG,Q,O,D,DM,AM,RE) }.
on_route("N757CC", 2, FlightsOnRoute):- FlightsOnRoute = #count {(F,REG) : REG="N757CC", R=2, p_a(F,R,A,REG,Q,O,D,DM,AM,RE) }.
on_route("N318DD", 1, FlightsOnRoute):- FlightsOnRoute = #count {(F,REG) : REG="N318DD", R=1, p_a(F,R,A,REG,Q,O,D,DM,AM,RE) }.
on_route("N318DD", 2, FlightsOnRoute):- FlightsOnRoute = #count {(F,REG) : REG="N318DD", R=2, p_a(F,R,A,REG,Q,O,D,DM,AM,RE) }.

% Add the number of flights on a route to the data:
p_a_r(Flight, Route, Aircraft, Registration, Quantity, Origin, Destination, DepartureMST, ArrivalMST, Result, FlightsOnRoute) :- on_route(Registration, Route, FlightsOnRoute), p_a(Flight, Route, Aircraft, Registration, Quantity, Origin, Destination, DepartureMST, ArrivalMST, Result).

% Try to ensure that each aircraft returns to its origin airport after flying to its destination by preventing odd numbers of flights on a route.
:- FlightsOnRoute == 1, p_a_r(Flight, Route, Aircraft, Registration, Quantity, Origin, Destination, DepartureMST, ArrivalMST, Result, FlightsOnRoute).
:- FlightsOnRoute == 3, p_a_r(Flight, Route, Aircraft, Registration, Quantity, Origin, Destination, DepartureMST, ArrivalMST, Result, FlightsOnRoute).
:- FlightsOnRoute == 5, p_a_r(Flight, Route, Aircraft, Registration, Quantity, Origin, Destination, DepartureMST, ArrivalMST, Result, FlightsOnRoute).

% However, some cases remain where there are missing return trips despite the even number of flights on a route.
% Try to prevent such impossible schedules (Flight 1: airport A -> airport B; same aircraft, flight 2: airport A -> airport B).
% Only "keep" all instances where the first and second origin-destination pair are not identical (including the direction):
schedule(F,R,A,REG,Q,O,D,DM,AM,RE,FOR,"keep"):- p_a_r(F,R,A,REG,Q,O,D,DM,AM,RE,FOR), p_a_r(F2,R2,A2,REG,Q2,O2,D2,DM2,AM2,RE2,FOR2), O != O2, D != D2.

#show schedule/12.

Overwriting logic.lp


## 4. Optimization

The next step aims at defining a way to optimize the schedule that was generated by the previous logic program. In other words, it tries to maximize the total profit of the schedule. Therefore, weak constraints are added via an optimization statement.

In [3]:
%%file optimization.lp

% Optimization:
% ============

% Actual profit sum:
actual_profit_sum(Profit) :- Profit = #sum {RE : schedule(F,R,A,REG,Q,O,D,DM,AM,RE,FOR,X)}.

#show actual_profit_sum/1.

% Maximize actual profit sum:
#maximize { Profit : actual_profit_sum(Profit) }.

Overwriting optimization.lp


In [4]:
! python -m clingo 0 data.lp logic.lp optimization.lp

pyclingo version 5.6.2
Reading from data.lp ...
Solving...
Answer: 1
schedule("WA 152",1,"B737","N737AA",2,"ATL","SLC","16:30","21:00",18750,2,"keep") schedule("WA 154",2,"B737","N737BB",2,"PHX","SLC","18:15","19:55",11500,2,"keep") schedule("WA 121",1,"B737","N737AA",2,"SLC","ATL","06:40","10:25",11250,2,"keep") schedule("WA 201",2,"B737","N737BB",2,"SLC","PHX","11:00","12:40",1875,2,"keep") schedule("WA 153",2,"B757","N757CC",1,"PHX","SLC","07:45","09:30",3500,2,"keep") schedule("WA 202",2,"B757","N757CC",1,"SLC","PHX","14:20","16:05",-6000,2,"keep") schedule("WA 151",1,"A318","N318DD",1,"ATL","SLC","11:15","15:35",6000,2,"keep") schedule("WA 122",1,"A318","N318DD",1,"SLC","ATL","17:00","22:30",12000,2,"keep") actual_profit_sum(58875)
Optimization: -58875
Answer: 2
schedule("WA 152",1,"B737","N737AA",2,"ATL","SLC","16:30","21:00",18750,2,"keep") schedule("WA 153",2,"B737","N737BB",2,"PHX","SLC","07:45","09:30",7000,4,"keep") schedule("WA 154",2,"B737","N737BB",2,"PHX","SLC","18:15","

We can see that there are five models and that at the last one of them, the optimum is found. The profit of this schedule is $77525.

## 5. Final result

As a last step, the final schedule that has been optimized for maximizing profit is shown. The values below represent the final schedule that maximizes the profit according to the constraints and objective function defined in earlier steps. 

The final schedule of each flight displays details like the flight number, route number, aircraft type, aircraft registration, quantity of the aircraft type in the fleet, origin, destination, departure time, arrival time, profit of the flight and the number of flights of the individual aircraft on the respective route that day.

In [5]:
%%file optimum.lp

% Optimum:
% =======

% Final schedule optimized for maximizing profit:
schedule("WA 154",2,"B737","N737AA",2,"PHX","SLC","18:15","19:55",11500,2,"keep").
schedule("WA 152",1,"B737","N737BB",2,"ATL","SLC","16:30","21:00",18750,2,"keep").
schedule("WA 201",2,"B737","N737AA",2,"SLC","PHX","11:00","12:40",1875,2,"keep").
schedule("WA 121",1,"B737","N737BB",2,"SLC","ATL","06:40","10:25",11250,2,"keep").
schedule("WA 151",1,"B757","N757CC",1,"ATL","SLC","11:15","15:35",1750,2,"keep").
schedule("WA 122",1,"B757","N757CC",1,"SLC","ATL","17:00","22:30",22800,2,"keep").
schedule("WA 153",2,"A318","N318DD",1,"PHX","SLC","07:45","09:30",4600,2,"keep").
schedule("WA 202",2,"A318","N318DD",1,"SLC","PHX","14:20","16:05",5000,2,"keep").

% Profit: 77525

Overwriting optimum.lp


## Bonus chapter: comparison

In this (trivial) step, we compare the profit of the final schedule obtained from the optimization step with the profit that would be obtained if the aircraft with the highest profit is assigned to each flight, without respecting any constraints. Of course, no schedule could ever be created this way as the resulting flight sequences would in many cases be physically impossible, but this comparison is interesting as it allows to estimate the "loss" caused by operational circumstances and constraints on the final schedule. The difference between this profit sum and the actual profit of the final schedule gives an indication of how much profit is lost due to the imposed constraints.

In [6]:
%%file comparison.lp

% Comparison:
% ==========

% Maximum profit per flight, disregarding all constraints:
max_profit(Flight, Aircraft, Result) :- profit(Flight, Aircraft, Result), Result = #max {R : profit("WA 121",A,R)}.
max_profit(Flight, Aircraft, Result) :- profit(Flight, Aircraft, Result), Result = #max {R : profit("WA 122",A,R)}.
max_profit(Flight, Aircraft, Result) :- profit(Flight, Aircraft, Result), Result = #max {R : profit("WA 151",A,R)}.
max_profit(Flight, Aircraft, Result) :- profit(Flight, Aircraft, Result), Result = #max {R : profit("WA 152",A,R)}.
max_profit(Flight, Aircraft, Result) :- profit(Flight, Aircraft, Result), Result = #max {R : profit("WA 153",A,R)}.
max_profit(Flight, Aircraft, Result) :- profit(Flight, Aircraft, Result), Result = #max {R : profit("WA 154",A,R)}.
max_profit(Flight, Aircraft, Result) :- profit(Flight, Aircraft, Result), Result = #max {R : profit("WA 201",A,R)}.
max_profit(Flight, Aircraft, Result) :- profit(Flight, Aircraft, Result), Result = #max {R : profit("WA 202",A,R)}.

% Maximum profit sum:
max_profit_sum(Result) :- Result = #sum {R : max_profit(F,A,R)}.

#show max_profit_sum/1.

Overwriting comparison.lp


In [7]:
! python -m clingo 0 data.lp comparison.lp

pyclingo version 5.6.2
Reading from data.lp ...
Solving...
Answer: 1
max_profit_sum(89550)
SATISFIABLE

Models       : 1
Calls        : 1
Time         : 0.007s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 0.007s


Based on this, we can calculate:
Actual profit / Maximum profit = 77525 / 89550 = 0.8657 <br>
The actual profit is or 86.6% of the unrealistic comparison value, so all of the constraints have only led to a minor "loss".